In [16]:
#Need to add parent directoy to sys.path to find 'metadataDB'
import sys
sys.path.append('../')

%matplotlib inline
# import matplotlib.pyplot as plt 
import time
# import numpy as np
# import scipy as sp
import re
from collections import Counter
import random

# Natural language processing toolkit
# To use this, run nltk.download() and download 'stopwords'
# from nltk.corpus import stopwords
# s=stopwords.words('english') + ['']

# Machine learning
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.svm import LinearSVC, SVC
# from sklearn.pipeline import Pipeline
# from sklearn import metrics

# SQL
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from metadataDB.declareDatabase import *
from sqlalchemy import or_, and_

engine = create_engine("sqlite:///../arXiv_metadata.db", echo=False)
Base.metadata.bind = engine
DBsession = sessionmaker(bind=engine)
session = DBsession()

In [17]:
query = session.query(Article).filter(or_(Article.title.like('%high?order%'),
                                          Article.title.like('%high?harmonic%'),
                                          Article.title.like('%ultrafast%'),
                                          Article.title.like('%femtosecond%'),
                                          Article.title.like('%attosecond%'),
                                          Article.title.like('%zeptosecond%')))
resultPRL = query.all()
abstract_all = [x.abstract for x in query.all()]
session.close_all()

In [18]:
abstract_train = abstract_all
print len(abstract_train)

1218


CountVectorizer might do a cleaner job.

In [19]:
class AbstractWriter:
    '''This class will take a specific n-gram and learn to write abstracts from a few words.'''
    _data = dict()
    _abstracts = []
    
    def __init__(self, ngram, randomize=False, seed=None, maxWords=200):
        self.ngram = ngram
        self.randomize = randomize
        self.seed = seed
        self.maxWords = maxWords
        if seed is not None:
            random.seed(seed)

    
    def fit(self, train):
        data_list = []
        for item in train:
            ind = 0
            textlist = self.__cleanup_text(item)
            self._abstracts.append(' '.join(textlist))
            while ind + self.ngram - 1 < len(textlist):
                currentTuple = tuple(textlist[ind:(ind+self.ngram-1)])
                nextWord = textlist[ind+self.ngram-1]
                data_list.append((tuple(textlist[ind:(ind+self.ngram-1)]),
                                        textlist[ind+self.ngram-1]))
                ind += 1
        data_list.sort()
        
        # Now that the list is sorted, convert it into a dictionary.
        # Since the list is sorted, each subsuquent entry is either new or a continuation of the previous one.

        previousKey = None
        for val in data_list:
            currentKey = val[0]
            if currentKey == previousKey:
                self._data[currentKey].append(val[1])
            else:
                self._data[currentKey] = [val[1]]
            previousKey = val[0]

    def find_next_word(self, phrase):
        '''phrase is a tuple of (typ. 3) words. Returns the next word.'''
        # Since data is sorted, it should be possible to search quickly. I'm not doing this properly.
        if phrase in self._data.keys():
            solution = self._data[phrase]
            
            # If there are more than one result, we either return the most popular option or
            # a random answer weighted by popularity.
            if self.randomize:
                return random.choice(solution)
            else:
                count = Counter(solution)
                return count.most_common(1)[0][0]
        else:
            return None
        
    def write_abstract(self, beginning):
        abstract = list(beginning)
        nextWord = ''
        ind = len(beginning) - self.ngram + 1
        while (nextWord is not None) and (ind < self.maxWords):
            nextWord = self.find_next_word(tuple(abstract[ind:(ind+self.ngram)]))
            if nextWord is None:
                break
            else:
                abstract.append(nextWord)
                ind += 1
        return ' '.join(abstract)
    
    def find_similar(self, abstract):
        '''Have I accidentally plagarized one of the articles?
        With too few choices, we will likely just copy an existing article.
        Returns the longest chunk highlighted in an existing article.'''
        first = 0
        last = 1
        abort = False
        longestChunk = ''
        longestChunkLength = 0

        totalLength = len(abstract.split())
        while first + longestChunkLength < totalLength:
            chunk = ' '.join(abstract.split()[first:last])
            if any(x for x in self._abstracts if chunk in x):
                if last - first + 1 > longestChunkLength:
                    longestChunk = chunk
                    longestChunkLength = last - first + 1
                last += 1
            else:
                first += 1
                last = first + 1
        
        best_abstract = (x for x in self._abstracts if longestChunk in x).next()
        return re.sub('(' + longestChunk + ')', '[[[' + longestChunk + ']]]', best_abstract)


    def __cleanup_text(self, text):
        # re from kaggle https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-wordsi
        # Remove punctuation. Keep periods.
        # 
        # New trick: replace numbers with ##, which we will later replace with different numbers.
        pattern1 = re.compile("[0-9]")
        pattern2 = re.compile("[^a-zA-Z#.]")
        # The previous pattern could make multiple spaces. Turn multiple spaces into a single space.
        pattern3 = re.compile("(\s{2,})")
        return pattern3.sub(" ",
                            pattern2.sub(" ",
                                         pattern1.sub("#",
                                                      text.lstrip().rstrip()))).split(' ')

Let's make a bunch of abstracts that start in a way similar to existing abstracts, and find if they accidentally plagarize anything.

In [20]:
def WriteAbstract(ngram):
    writer = AbstractWriter(ngram=ngram, randomize=True, seed=42)
    writer.fit(abstract_train)
    for i in range(0,5):
        phrase = random.choice(writer._abstracts).split()[0:(ngram-1)]
        currentAbstract = writer.write_abstract(tuple(phrase))
        print 'New abstract: ' + currentAbstract
        print ''
        print 'Existing abstract: ' + writer.find_similar(currentAbstract)
        print ''
    return writer

In [21]:
WriteAbstract(4)

New abstract: This letter is the first demonstration of visible ###nm third harmonic generation in gas. This radiation is monochromatized by a time delayed frequency doubled pulse from nascent LIPSS formation induced by the pump transition.

Existing abstract: Time resolved diffraction microscopy technique has been used to observe the formation of laser induced periodic surface structures LIPSS from the interaction of a single femtosecond laser pulse pump with a nano scale groove mechanically formed on a single crystal Cu substrate. The interaction dynamics # #### ps was captured by diffracting [[[a time delayed frequency doubled pulse from nascent LIPSS formation induced by the pump]]] with an infinity conjugate microscopy setup. The LIPSS ripples are observed to form sequentially outward from the groove edge with the first one forming after ## ps. A # D analytical model of electron heating and surface plasmon polariton SPP excitation induced by the interaction of incoming laser pulse

<__main__.AbstractWriter instance at 0x106f73c68>

In [22]:
WriteAbstract(5)

New abstract: Subpicosecond wavelength non degenerate differential transmission DT was used to observe the carrier relaxation mechanism in GaN based quantum well QW with and without metal nanoparticles MNPs in it. The spontaneous emission dominates the stimulated emission for above the GaN band edge excitation energy. We observed long decay times for Au embedded sample and short decay times for Ag embedded sample with respect to reference sample.

Existing abstract: [[[Subpicosecond wavelength non degenerate differential transmission DT was used to observe the carrier relaxation mechanism in GaN based quantum well QW with and without metal nanoparticles MNPs in it. The spontaneous emission dominates the stimulated emission for above the GaN band edge excitation energy. We observed long decay times for Au embedded sample and short decay times for Ag embedded sample with respect to reference]]] sample.

New abstract: We present fully ab initio simulations of attosecond streaking for effe

<__main__.AbstractWriter instance at 0x1074e3e18>

In [23]:
WriteAbstract(6)

New abstract: The feasibility of generation of bright ultrashort gamma ray pulses is demonstrated in the interaction of a relativistic electron bunch with a counterpropagating tightly focused superstrong laser beam in the radiation dominated regime. The Compton scattering spectra of gamma radiation are investigated using a semiclassical description for the electron dynamics in the molecular ion can be understood via the phase difference accumulated between the Coulomb wells at the two protons. Modeling of the dynamics using a simple two state system leads us to an explanation for the sometimes counter intuitive dynamics of an electron opposing the classical force of the electric field on the electron.

Existing abstract: We analyze the attosecond electron dynamics in hydrogen molecular ion driven by an external intense laser field using ab initio numerical simulations of the corresponding time dependent Schr o dinger equation and Bohmian trajectories. To this end we employ a one dimens

<__main__.AbstractWriter instance at 0x108787b90>